# Recitation - 5
___

* Custom Dataset & DataLoader
* Torchvision ImageFolder Dataset
* Residual Block
* CNN model with Residual Block

## Imports

In [1]:
import os
import numpy as np
from PIL import Image

import torch
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

## Custom DataSet with DataLoader
___
We have used a subset of MNIST

In [2]:
class ImageDataset(Dataset):
    def __init__(self, file_list, target_list):
        self.file_list = file_list
        self.target_list = target_list
        self.n_class = len(list(set(target_list)))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        img = torchvision.transforms.ToTensor()(img)
        label = self.target_list[index]
        return img, label

#### Parse the given directory to accumulate all the images

In [3]:
def parse_data(datadir):
    img_list = []
    ID_list = []
    for root, directories, filenames in os.walk(datadir):  #root: median/1
        for filename in filenames:
            if filename.endswith('.png'):
                filei = os.path.join(root, filename)
                img_list.append(filei)
                ID_list.append(root.split('/')[-1])

    # construct a dictionary, where key and value correspond to ID and target
    uniqueID_list = list(set(ID_list))
    class_n = len(uniqueID_list)
    target_dict = dict(zip(uniqueID_list, range(class_n)))
    label_list = [target_dict[ID_key] for ID_key in ID_list]

    print('{}\t\t{}\n{}\t\t{}'.format('#Images', '#Labels', len(img_list), len(set(label_list))))
    return img_list, label_list, class_n

In [4]:
img_list, label_list, class_n = parse_data('mnist/training')

#Images		#Labels
5000		10


In [5]:
print(img_list[1888])

mnist/training/7/40510.png


In [6]:
trainset = ImageDataset(img_list, label_list)

In [7]:
train_data_item, train_data_label = trainset.__getitem__(0)

In [8]:
print('data item shape: {}\t data item label: {}'.format(train_data_item.shape, train_data_label))

data item shape: torch.Size([1, 28, 28])	 data item label: 5


In [9]:
dataloader = DataLoader(trainset, batch_size=10, shuffle=True, num_workers=1, drop_last=False)

## Torchvision DataSet and DataLoader

In [10]:
imageFolder_dataset = torchvision.datasets.ImageFolder(root='mnist/training/', 
                                                       transform=torchvision.transforms.ToTensor())

In [11]:
imageFolder_dataloader = DataLoader(imageFolder_dataset, batch_size=10, shuffle=True, num_workers=1)

In [12]:
print(imageFolder_dataset.__len__(), len(imageFolder_dataset.classes))

5000 10


## Residual Block

Resnet: https://arxiv.org/pdf/1512.03385.pdf

Here is a basic usage of shortcut in Resnet

In [13]:
# This is the simplest possible residual block, with only one CNN layer.
# Looking at the paper, you can extend this block to have more layers, bottleneck, grouped convs (from shufflenet), etc.
# Or even look at more recent papers like resnext, regnet, resnest, senet, etc.
class SimpleResidualBlock(nn.Module):
    def __init__(self, channel_size, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(channel_size, channel_size, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channel_size)
        if stride == 1:
            self.shortcut = nn.Identity()
        else:
            self.shortcut = nn.Conv2d(channel_size, channel_size, kernel_size=1, stride=stride)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        
        shortcut = self.shortcut(x)
        
        out = self.relu(out + shortcut)
        
        return out

In [14]:
# This has hard-coded hidden feature sizes.
# You can extend this to take in a list of hidden sizes as argument if you want.
class ClassificationNetwork(nn.Module):
    def __init__(self, in_features, num_classes):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Conv2d(in_features, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            SimpleResidualBlock(64),
            SimpleResidualBlock(64),
            SimpleResidualBlock(64),
            SimpleResidualBlock(64),
            nn.AdaptiveAvgPool2d((1, 1)), # For each channel, collapses (averages) the entire feature map (height & width) to 1x1
            nn.Flatten(), # the above ends up with batch_size x 64 x 1 x 1, flatten to batch_size x 64
        )
        self.linear = nn.Linear(64, num_classes)
        
    def forward(self, x, return_embedding=False):
        embedding = self.layers(x) 
        
        if return_embedding:
            return embedding
        else:
            return self.linear(embedding)       

In [15]:
train_dataset = torchvision.datasets.ImageFolder(root='mnist/training/', 
                                                 transform=torchvision.transforms.ToTensor())
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, 
                                               shuffle=True, num_workers=8)

dev_dataset = torchvision.datasets.ImageFolder(root='mnist/testing/', 
                                               transform=torchvision.transforms.ToTensor())
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=128, 
                                             shuffle=False, num_workers=8)

In [16]:
numEpochs = 10
in_features = 3 # RGB channels

learningRate = 5e-2
weightDecay = 5e-5

num_classes = len(train_dataset.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

network = ClassificationNetwork(in_features, num_classes)
network = network.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)

In [17]:
# Train!
for epoch in range(numEpochs):
    
    # Train
    network.train()
    avg_loss = 0.0
    for batch_num, (x, y) in enumerate(train_dataloader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = network(x)

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()

        if batch_num % 10 == 9:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, avg_loss/50))
            avg_loss = 0.0
    
    # Validate
    network.eval()
    num_correct = 0
    for batch_num, (x, y) in enumerate(dev_dataloader):
        x, y = x.to(device), y.to(device)
        outputs = network(x)
        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
        
    print('Epoch: {}, Validation Accuracy: {:.2f}'.format(epoch, num_correct / len(dev_dataset)))

Epoch: 0	Batch: 10	Avg-Loss: 0.4558
Epoch: 0	Batch: 20	Avg-Loss: 0.3748
Epoch: 0	Batch: 30	Avg-Loss: 0.3292
Epoch: 0	Batch: 40	Avg-Loss: 0.2966
Epoch: 0, Validation Accuracy: 0.19
Epoch: 1	Batch: 10	Avg-Loss: 0.3116
Epoch: 1	Batch: 20	Avg-Loss: 0.2600
Epoch: 1	Batch: 30	Avg-Loss: 0.2220
Epoch: 1	Batch: 40	Avg-Loss: 0.2091
Epoch: 1, Validation Accuracy: 0.25
Epoch: 2	Batch: 10	Avg-Loss: 0.2039
Epoch: 2	Batch: 20	Avg-Loss: 0.1892
Epoch: 2	Batch: 30	Avg-Loss: 0.1488
Epoch: 2	Batch: 40	Avg-Loss: 0.1349
Epoch: 2, Validation Accuracy: 0.44
Epoch: 3	Batch: 10	Avg-Loss: 0.1312
Epoch: 3	Batch: 20	Avg-Loss: 0.1020
Epoch: 3	Batch: 30	Avg-Loss: 0.0960
Epoch: 3	Batch: 40	Avg-Loss: 0.0910
Epoch: 3, Validation Accuracy: 0.61
Epoch: 4	Batch: 10	Avg-Loss: 0.0961
Epoch: 4	Batch: 20	Avg-Loss: 0.0848
Epoch: 4	Batch: 30	Avg-Loss: 0.0691
Epoch: 4	Batch: 40	Avg-Loss: 0.0577
Epoch: 4, Validation Accuracy: 0.84
Epoch: 5	Batch: 10	Avg-Loss: 0.0703
Epoch: 5	Batch: 20	Avg-Loss: 0.0548
Epoch: 5	Batch: 30	Avg-Loss:

In [28]:
# Let's try cosine similarity

compute_sim = nn.CosineSimilarity(dim=0)

img_a = dev_dataset[0][0] # this is class 0
img_b = dev_dataset[1][0] # this is also class 0
img_c = dev_dataset[51][0] # this is class 1
img_d = dev_dataset[451][0] # this is class 9

In [32]:
network.eval()
feats_a = network(img_a.cuda().unsqueeze(0), return_embedding=True).squeeze(0)
feats_b = network(img_b.cuda().unsqueeze(0), return_embedding=True).squeeze(0)
feats_c = network(img_c.cuda().unsqueeze(0), return_embedding=True).squeeze(0)
feats_d = network(img_d.cuda().unsqueeze(0), return_embedding=True).squeeze(0)

In [38]:
print("CS b/n two images of class 0: {:.4f}".format(compute_sim(feats_a, feats_b)))
print("CS b/n an image of class 0 and image of class 1: {:.4f}".format(compute_sim(feats_a, feats_c)))
print("CS b/n an image of class 0 and image of class 9: {:.4f}".format(compute_sim(feats_a, feats_d)))

CS b/n two images of class 0: 0.9862
CS b/n an image of class 0 and image of class 1: 0.8086
CS b/n an image of class 0 and image of class 9: 0.8582


In [39]:
# We see that CS between images of same class are highest
# CS between 0 and 1 is lower, while CS between 0 and 9 are closer (prolly because they both have circles)